In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os



# Import API key
from api_keys import g_key

In [8]:
file = "output/my_combined.csv"

county_df = pd.read_csv(file)
county_df.head()

file2 = "Resources/countycoordinates.xlsx"

coord_df = pd.read_excel(file2)
clean_coord = coord_df[["State", "County", "Latitude", "Longitude"]]
clean_coord

data_loc = pd.DataFrame.merge(county_df, clean_coord, on=["State", "County"])



#data_loc.to_csv('output/clean_with_lat_long.csv', index = False)

data_loc

,State,County,POP2010,OHU2010,PovertyRate,MedianFamilyIncome,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,...,# Deaths,Years of Potential Life Lost Rate,% Obese,% Fair/Poor,# Diabetics,# Violent Crimes,# Unemployed,% Children in Poverty,Latitude,Longitude
0,Alabama,Autauga,54571,20221,13.858333,61082.916667,2,2,1,2,...,830.0,9158.0,34,18,711.0,145.0,1327.0,19.0,32.534920,-86.642749
1,Alabama,Baldwin,182265,73180,14.358065,60664.935484,3,3,3,2,...,2573.0,7394.0,27,16,2615.0,415.0,4791.0,20.0,30.660970,-87.749840
2,Alabama,Barbour,27457,9820,24.533333,43123.000000,3,3,1,2,...,413.0,8573.0,44,25,525.0,85.0,771.0,45.0,31.869603,-85.393197
3,Alabama,Bibb,22915,7953,16.025000,43362.500000,0,0,0,1,...,407.0,11766.0,40,20,333.0,33.0,561.0,29.0,32.998644,-87.126439
4,Alabama,Blount,57322,21578,17.911111,52136.000000,0,0,0,2,...,906.0,9333.0,35,19,705.0,123.0,1316.0,22.0,33.980867,-86.567371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3090,Wyoming,Sweetwater,43806,16475,11.841667,81835.083333,0,0,0,2,...,445.0,6878.0,32,16,469.0,159.0,1053.0,10.0,41.659496,-108.879431
3091,Wyoming,Teton,21294,8973,8.100000,89555.250000,0,0,0,0,...,116.0,3728.0,13,12,104.0,NaN,558.0,8.0,43.934637,-110.589810
3092,Wyoming,Uinta,21118,7668,14.166667,65312.666667,0,0,0,0,...,234.0,7660.0,28,15,243.0,8.0,475.0,12.0,41.287492,-110.547562
3093,Wyoming,Washakie,8533,3492,14.133333,62213.666667,0,1,0,0,...,97.0,7294.0,28,14,156.0,7.0,174.0,16.0,43.904997,-107.682861


In [14]:
file = "output/clean_with_lat_long.csv"

longlat_df = pd.read_csv(file)
longlat_df.head()

file2 = "Resources/Medicare_Enrollment_2017.csv"

health_df = pd.read_csv(file2)

clean_data2 = pd.DataFrame.merge(longlat_df, health_df, on=["State", "County"])



#clean_data2.to_csv('output/clean_data2.csv', index = False)

clean_data2

,State,County,POP2010,OHU2010,PovertyRate,MedianFamilyIncome,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,...,Years of Potential Life Lost Rate,% Obese,% Fair/Poor,# Diabetics,# Violent Crimes,# Unemployed,% Children in Poverty,Latitude,Longitude,Original Medicare
0,Alabama,Autauga,54571,20221,13.858333,61082.916667,2,2,1,2,...,9158.0,34,18,711.0,145.0,1327.0,19.0,32.534920,-86.642749,5784
1,Alabama,Baldwin,182265,73180,14.358065,60664.935484,3,3,3,2,...,7394.0,27,16,2615.0,415.0,4791.0,20.0,30.660970,-87.749840,28388
2,Alabama,Barbour,27457,9820,24.533333,43123.000000,3,3,1,2,...,8573.0,44,25,525.0,85.0,771.0,45.0,31.869603,-85.393197,4372
3,Alabama,Bibb,22915,7953,16.025000,43362.500000,0,0,0,1,...,11766.0,40,20,333.0,33.0,561.0,29.0,32.998644,-87.126439,2480
4,Alabama,Blount,57322,21578,17.911111,52136.000000,0,0,0,2,...,9333.0,35,19,705.0,123.0,1316.0,22.0,33.980867,-86.567371,6070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3088,Wyoming,Sweetwater,43806,16475,11.841667,81835.083333,0,0,0,2,...,6878.0,32,16,469.0,159.0,1053.0,10.0,41.659496,-108.879431,5641
3089,Wyoming,Teton,21294,8973,8.100000,89555.250000,0,0,0,0,...,3728.0,13,12,104.0,NaN,558.0,8.0,43.934637,-110.589810,3122
3090,Wyoming,Uinta,21118,7668,14.166667,65312.666667,0,0,0,0,...,7660.0,28,15,243.0,8.0,475.0,12.0,41.287492,-110.547562,2804
3091,Wyoming,Washakie,8533,3492,14.133333,62213.666667,0,1,0,0,...,7294.0,28,14,156.0,7.0,174.0,16.0,43.904997,-107.682861,1812


In [16]:
for col in clean_data2.columns:
    print(col)

State
County
POP2010
OHU2010
PovertyRate
MedianFamilyIncome
LILATracts_1And10
LILATracts_halfAnd10
LILATracts_1And20
LILATracts_Vehicle
LAPOP1_10
LAPOP05_10
LAPOP1_20
LALOWI1_10
LALOWI05_10
LALOWI1_20
lapophalf
TractKids
TractSeniors
TractWhite
TractBlack
TractAsian
TractNHOPI
TractAIAN
TractOMultir
TractHispanic
TractHUNV
TractSNAP
Food Environment Index
# Deaths
Years of Potential Life Lost Rate
% Obese
% Fair/Poor
# Diabetics
# Violent Crimes
# Unemployed
% Children in Poverty
Latitude
Longitude
Original Medicare


In [11]:
locations = data_loc[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
food_env = data_loc["Food Environment Index"].astype(float)

In [18]:
# Plot Heatmap
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=food_env, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=.25)


# Add layer
fig.add_layer(heat_layer)

#Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [20]:
montana = clean_data2.loc[clean_data2["State"] == "Montana",:]

montana.head()

,State,County,POP2010,OHU2010,PovertyRate,MedianFamilyIncome,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,...,Years of Potential Life Lost Rate,% Obese,% Fair/Poor,# Diabetics,# Violent Crimes,# Unemployed,% Children in Poverty,Latitude,Longitude,Original Medicare
1585,Montana,Beaverhead,9246,4014,14.566667,53953.0,2,2,2,1,...,6824.0,24,12,126.0,10.0,171.0,21.0,45.132700,-112.898820,1841
1586,Montana,Big Horn,12865,4004,31.220000,46813.2,4,4,1,1,...,15117.0,39,22,193.0,28.0,372.0,40.0,45.423382,-107.489853,1607
1587,Montana,Blaine,6491,2357,29.800000,38925.5,1,1,1,1,...,11508.0,36,20,100.0,10.0,120.0,40.0,48.432676,-108.958602,1004
1588,Montana,Broadwater,5612,2347,11.250000,48507.5,0,0,0,0,...,10532.0,30,12,64.0,20.0,124.0,16.0,46.332424,-111.495487,1111
1589,Montana,Carbon,10078,4571,11.480000,60412.6,1,1,0,0,...,7097.0,24,11,131.0,20.0,200.0,15.0,45.227386,-109.028016,1993


In [22]:
locations2 = montana[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
food_env2 = montana["Food Environment Index"].astype(float)

In [23]:
# Plot Heatmap
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations2, weights=food_env2, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=.25)


# Add layer
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))